In [112]:
import pandas as pd

zero_shot_data = pd.read_csv('zero_shot_answers.csv')
chain_of_thought_data = pd.read_csv('chain_of_thought_answers.csv')

In [113]:
class Data:
    
	def __init__(self, file_name):
		self.data = pd.read_csv(file_name)
		self.regex = r'Option '
		self.regex = r'(?i)([1-9A-Da-d])'+self.regex[::-1]

		self.process_data()
		self.get_accuracy()
		
	def process_data(self):
		self.extract_options()
		self.add_correct_answers()

		self.data.drop(columns=['gemma', 'phi', 'llama'], inplace=True)

		self.process_options()

	def extract_options(self):

		self.data['gemma'] = self.data['gemma'].str[::-1]
		self.data['phi'] = self.data['phi'].str[::-1]
		self.data['llama'] = self.data['llama'].str[::-1]

		self.data['gemma_option'] = self.data['gemma'].str.extract(self.regex)
		self.data['phi_option'] =   self.data['phi'].str.extract(self.regex)
		self.data['llama_option'] = self.data['llama'].str.extract(self.regex)

		# replace A-D with 1-4
		self.data['gemma_option'] = self.data['gemma_option'].replace({'A':1, 'B':2, 'C':3, 'D':4})
		self.data['phi_option'] = self.data['phi_option'].replace({'A':1, 'B':2, 'C':3, 'D':4})
		self.data['llama_option'] = self.data['llama_option'].replace({'A':1, 'B':2, 'C':3, 'D':4})

		#replace a-d with 1-4
		self.data['gemma_option'] = self.data['gemma_option'].replace({'a':1, 'b':2, 'c':3, 'd':4})
		self.data['phi_option'] = self.data['phi_option'].replace({'a':1, 'b':2, 'c':3, 'd':4})
		self.data['llama_option'] = self.data['llama_option'].replace({'a':1, 'b':2, 'c':3, 'd':4})
	
	def add_correct_answers(self):
		hgf_dataset = pd.read_parquet("hf://datasets/cais/mmlu/college_mathematics/test-00000-of-00001.parquet")
		self.data['correct_answer'] = hgf_dataset['answer']
		self.data['correct_answer'] = self.data['correct_answer'] + 1

	def process_options(self):
		self.data.fillna(0, inplace=True)
		self.data = self.data.astype(int)


	def get_accuracy(self):
		print("Gemma accuracy: ", (self.data['gemma_option'] == self.data['correct_answer']).mean())
		print("Phi accuracy: ", (self.data['phi_option'] == self.data['correct_answer']).mean())
		print("Llama accuracy: ", (self.data['llama_option'] == self.data['correct_answer']).mean())


In [114]:
Data('zero_shot_answers.csv')

Gemma accuracy:  0.25
Phi accuracy:  0.28
Llama accuracy:  0.35


In [115]:
Data('chain_of_thought_answers.csv')

Gemma accuracy:  0.29
Phi accuracy:  0.41
Llama accuracy:  0.37


In [7]:
Data('react_answers.csv')

Gemma accuracy:  0.12
Phi accuracy:  0.32
Llama accuracy:  0.31
